In [4]:
# setup
import sqlite3
print('All good')

All good


### 1. Player

In [3]:
def player_upsert():
    conn = None
    cursor = None

    
    conn = sqlite3.connect('../databases/player.db')
    cursor = conn.cursor()

    cursor.execute('insert into player default values')

    conn.commit()
    conn.close()

### 7. Move

In [48]:
# PARAMETER DEFAULT VALUES
DEFAULT_DESCRIPTION = '' # (okay if it's empty)
DEFAULT_ATTACK_TYPE = ''
DEFAULT_BASE_DAMAGE = -10
DEFAULT_COST = '{}'
DEFAULT_EFFECTS = '{"null": "null"}'

def move_upsert(move_name, description = '', attack_type = '', base_damage = -10, cost = '{}', effects = '{"null": "null"}', force_insert = False):
    conn = None
    cursor = None

    try:
        conn = sqlite3.connect('../databases/move.db')
        cursor = conn.cursor()

        action_type = '' # we need to know whether we're going to be inserting or updating a record into the table
        cursor.execute(f'select count(*) from move where move_name = \"{move_name}\"')

        if cursor.fetchall()[0][0] > 0:
            action_type = 'upsert'
        else:
            action_type = 'insert'

        # action type is insert but not all parameters are populated (except description)
        if action_type == 'insert' and (attack_type == DEFAULT_ATTACK_TYPE or base_damage == DEFAULT_BASE_DAMAGE or cost == DEFAULT_COST or effects == DEFAULT_EFFECTS):
            print('All parameters need to be populated in order to insert into move.db')
            return # exits the function
        
        # action type is insert and all parameters are populated
        if action_type == 'insert' and not (attack_type == DEFAULT_ATTACK_TYPE or base_damage == DEFAULT_BASE_DAMAGE or cost == DEFAULT_COST or effects == DEFAULT_EFFECTS):
            # print(f'Inserting move {move_name}')
            cursor.execute(f'''
                insert into move (move_name, description, attack_type, base_damage, cost, effects)
                values (\'{move_name}\', \'{description}\', \'{attack_type}\', {base_damage}, \'{cost}\', \'{effects}\')
            ''')

        # action type is upsert and force insert is false
        if action_type == 'upsert' and force_insert == False:
            # first getting current values for each attribute
            cursor.execute(f'select * from move where move_name = \"{move_name}\"')
            result = cursor.fetchall()[0]
            curr_description = result[2]
            curr_attack_type = result[3]
            curr_base_damage = result[4]
            curr_cost = result[5]
            curr_effects = result[6]

            # only update the attributes we specified
            if description != DEFAULT_DESCRIPTION:
                curr_description = description
            if attack_type != DEFAULT_ATTACK_TYPE:
                curr_attack_type = attack_type
            if base_damage != DEFAULT_BASE_DAMAGE:
                curr_base_damage = base_damage
            if cost != DEFAULT_COST:
                curr_cost = cost
            if effects != DEFAULT_EFFECTS:
                curr_effects = effects

            # ready to update row
            cursor.execute(f'''
                update move
                set
                    description = \'{curr_description}\',
                    attack_type = \'{curr_attack_type}\',
                    base_damage = {curr_base_damage},
                    cost = \'{curr_cost}\',
                    effects = \'{curr_effects}\'
                where move_name = \'{move_name}\'
            ''')

        conn.commit()
    except Exception as e:
        print(f'SQL ERROR - {e}')
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [ ]:
# test ground here

# move_upsert('Vine Whip', '', 'grass', 40, '{"grass": 1, "colorless": 1}', '{}')
# move_upsert('Razor Leaf', '', 'grass', 60, '{"grass": 1, "colorless": 2}', '{}')
# move_upsert('Mega Drain', 'Heal 30 damage from this Pokemon', 'grass', 80, '{"grass": 2, "colorless": 2}', '{"heal_self": 30}')
move_upsert('Giant Bloom') # upsert works!

In [14]:
# test ground here

print('{"null", "null"}')

{"null", "null"}
